In [40]:
using CSV, MosekTools, DataFrames, JuMP, Random, LinearAlgebra, MathOptInterface, Mosek

# Loading data

In [2]:
df_train = CSV.read("../data/3. Housing/housing_train.csv")
df_test = CSV.read("../data/3. Housing/housing_test.csv")
X_train = convert(Matrix, df_train[:, 1:13])
y_train = df_train[:, 14]
X_test = convert(Matrix, df_test[:, 1:13])
y_test = df_test[:, 14]
println("Loaded and preprocessed Housing data for regression")

Loaded and preprocessed Housing data for regression


In [11]:
MathOptInterface.ExponentialCone()

MathOptInterface.ExponentialCone()

# Raw EVaR Implementation

In [38]:
function EVaR_raw_square_loss(X, y, α) # α is the training size in the stable case (1-α) is the validation size
    n,d = size(X)
    model_evar = Model(with_optimizer(Mosek.Optimizer))
    @variable(model_evar, s >= 0)
    @variable(model_evar, u[1:n] >= 0)
    @variable(model_evar, Z[1:n])
    @variable(model_evar, β[1:d])
    @variable(model_evar, t)
    
    @NLconstraint(
        model_evar,
        [i=1:n],
        Z[i] == (sum(β[j]*X[i,j] for j=1:d) - y[i])^2
    )
    @constraint(
        model_evar,
        sum(u[i] for i=1:n) <= n*α
    )
    @constraint(
        model_evar,
        [i=1:n],
        [u[i], 1, (Z[i]-t)/s] in MathOptInterface.ExponentialCone()
    )
    
    @objective(
        model_evar, 
        Min, 
        t
    )
    optimize!(model_evar)
    β_optimal = JuMP.value.(β)
    return β_optimal
end

EVaR_raw_square_loss (generic function with 1 method)

In [39]:
β_optimal = EVaR_raw_square_loss(X_train, y_train, 0.8)

ErrorException: Cannot divide affine expression by a variable